<h1>Sunku Bhanu Kedhaar Nath   -Z1974769</h1>

In [16]:
import aiohttp
import asyncio
import os
import time
import shutil
import nest_asyncio; nest_asyncio.apply()
import pandas as pd
import numpy as np
import zipfile

if not os.path.exists('artist-data'):
    os.mkdir('artist-data')

files =[]
files_l =[]
file_names = []

async def download_site(session, url):
    
    file = r'artist-data/' + url[50 : 99]
    #print(url[50:99])
    files.append(file)
    if os.path.exists(file):
        print(f'{file} already downloded')
    else:
        async with session.get(url) as response:
            with open(file,"wb") as d_file:
                async for content in response.content.iter_chunked(1024):
                    d_file.write(content)


def extract_files():
    for f in files:
        #print(f, "not here")
        if os.path.exists(f[:len(f)-4]):
            print(f[13:len(f)-4], "already extracted")
        else:
            #print(f, "here")
            with zipfile.ZipFile(f) as zf:
                zf.extractall('artist-data/')
            
#            shutil.unpack_archive(f, 'artist-data/')



def delete_files():
     for file in files:
        print(file[13:len(file)],"deleted Here")
        if os.path.exists("artist-data/" + file[13:len(file)]):
            shutil.rmtree("artist-data/" + file[13:len(file)])

            


async def download_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for site in sites:
            task = asyncio.ensure_future(download_site(session, site))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)
        
    
if __name__ == '__main__':
    tuples = [(75,79), (80,99), (100,119), (120,199), (200,249), (250,999)]
    sites = []
    for x, y in tuples:
        a = f'https://faculty.cs.niu.edu/~dakoop/cs503-2022fa/a7/{x}_million_to_{y}_million.zip'
        sites.append(a)
    s_time = time.time()
    asyncio.get_event_loop().run_until_complete(download_sites(sites))
    duration = time.time() - s_time
    print(f"downloded {len(sites)} in {duration}")
    extract_files()
    #delete_files()
    

artist-data//75_million_to_79_million.zip already downloded
artist-data//80_million_to_99_million.zip already downloded
artist-data//100_million_to_119_million.zip already downloded
artist-data//120_million_to_199_million.zip already downloded
artist-data//200_million_to_249_million.zip already downloded
artist-data//250_million_to_999_million.zip already downloded
downloded 6 in 0.00189208984375
75_million_to_79_million already extracted
80_million_to_99_million already extracted
100_million_to_119_million already extracted
120_million_to_199_million already extracted
200_million_to_249_million already extracted
250_million_to_999_million already extracted


In [10]:
from pathlib import Path
import glob
def find_matching_files():
    p = Path('.')
    file_names = []
    files_l = []
    for file_path in p.glob('artist-data/**/*.npy'):
        file_names.append(file_path.name)
        files_l.append(str(file_path))
        # print(file_path, "bucabac", file_path.name)
        
    # files_l = glob.glob('artist-data/**/*.npy', recursive = True)
    
    
    return files_l, file_names
    

files_l, file_names = find_matching_files()
files_l, file_names


(['artist-data/75_million_to_79_million/United States/kenny-g.npy',
  'artist-data/75_million_to_79_million/United States/alabama.npy',
  'artist-data/75_million_to_79_million/United States/luke-bryan.npy',
  'artist-data/75_million_to_79_million/United States/imagine-dragons.npy',
  'artist-data/75_million_to_79_million/United States/gloria-estefan.npy',
  'artist-data/75_million_to_79_million/Other/the-weeknd.npy',
  'artist-data/100_million_to_119_million/United States/george-strait.npy',
  'artist-data/100_million_to_119_million/United States/janet-jackson.npy',
  'artist-data/100_million_to_119_million/United Kingdom/george-michael.npy',
  'artist-data/100_million_to_119_million/Other/nicki-minaj.npy',
  'artist-data/80_million_to_99_million/United States/lionel-richie.npy',
  'artist-data/80_million_to_99_million/United States/post-malone.npy',
  'artist-data/80_million_to_99_million/United States/ariana-grande.npy',
  'artist-data/80_million_to_99_million/United States/flo-rida.

In [11]:
#this one works too (without threading)


# from concurrent.futures import ThreadPoolExecutor
# import threading
# my_lock = threading.Lock()

# df2 = pd.DataFrame(columns = ["Date", "Views", "Artist"])

# def process_files(file, file_name):
#     artist_list = []
#     df1 = pd.DataFrame()
#     #dates_and_views1 = np.array()
#     dates_and_views = np.load(file)
#     df = pd.DataFrame(dates_and_views.T, columns = ['Date','Views'])
#     artist_name = file_name[:len(file_name)-4].replace("-"," ")
#     df['Artist'] = artist_name.title()
    
    
#     return df
    
#     # for file, file_name in zip(files_l,file_names):
#     #     df1 = process_files(file, file_name)
#     #     #print(df1)
#     #     # df2 = pd.concat([df2,df1.to_frame()], ignore_index = True)
#     #     df2 = pd.concat([df2,df1], ignore_index = True)
#     # print(df)
#     # print(artist_name)
#     # print()
#     # c = pd.Series(dates_and_views, index = np.arange(len(dates_and_views)))


    
# for file, file_name in zip(files_l,file_names):
#     with ThreadPoolExecutor(max_workers = 5) as executor:
#         df1 = process_files(file, file_name)
#         #print(df1)
#         # df2 = pd.concat([df2,df1.to_frame()], ignore_index = True)
#         df2 = pd.concat([df2,df1], ignore_index = True)
# print(df2)

In [12]:
# with threading
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import threading

my_lock = threading.Lock()


def processing_files(files_l, file_names):
    artist_li =[]
    # print(files_l)
    # with my_lock:
    with my_lock:
        for file, file_name in zip(files_l, file_names):
            df1 = pd.DataFrame()
            #print(file)
            date_and_view = np.load(file)
            df = pd.DataFrame(date_and_view.T,columns = ['Date','Views'])
            df["Artist"] = file_name[:len(file_name)-4].replace("-"," ")
            artist_li.append(df)
        # print("here")
        # print(artist_li)
    return artist_li


with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
    res = executor.submit(processing_files, files_l, file_names)
    # print([all_artist_data], "here")

# print(res.result()) 
df2 = pd.concat(res.result(), ignore_index = True)
print(df2)
# res = list(res)
# print(res)
# all_data = list(all_artist_data)
# print(all_data)
# all_data1 = []

#df2 = pd.concat(all_artist_data, ignore_index = True)

# df2 = pd.concat(res, ignore_index = True)

#print(df2)

    
        


         Date   Views         Artist
0    20211101   47615        kenny g
1    20211201  143498        kenny g
2    20220101   48071        kenny g
3    20220201   44062        kenny g
4    20220301   37992        kenny g
..        ...     ...            ...
379  20220601  710633  justin bieber
380  20220701  364010  justin bieber
381  20220801  379012  justin bieber
382  20220901  394924  justin bieber
383  20221001  398620  justin bieber

[384 rows x 3 columns]


In [13]:
grouped = df2.groupby('Date')
print(type(grouped))
for name, group in grouped:
    print(name)
    print(group)
    
    group.to_csv('artist-data/' + str(name) + '.csv.gz', index = False,compression="gzip")

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
20211101
         Date    Views           Artist
0    20211101    47615          kenny g
12   20211101    20838          alabama
24   20211101    70958       luke bryan
36   20211101   118801  imagine dragons
48   20211101    60161   gloria estefan
60   20211101   297587       the weeknd
72   20211101    92144    george strait
84   20211101   222100    janet jackson
96   20211101   440816   george michael
108  20211101   244394      nicki minaj
120  20211101    85841    lionel richie
132  20211101   178331      post malone
144  20211101   723490    ariana grande
156  20211101    33682         flo rida
168  20211101   199400           r.e.m.
180  20211101   365917          madonna
192  20211101   400343          rihanna
204  20211101  1586887     taylor swift
216  20211101   277253  whitney houston
228  20211101   184987      celine dion
240  20211101   401239       bruno mars
252  20211101   257626       katy perry
264  20211101   

In [14]:
# # with-out Threading


# from concurrent.futures import ThreadPoolExecutor
# import concurrent.futures

# my_lock = threading.Lock()


# def processing_files(files_l, file_names):
#     artist_li =[]
#     # print(files_l)
#     for file, file_name in zip(files_l, file_names):
#         df1 = pd.DataFrame()
#         #print(file)
#         date_and_view = np.load(file)
#         df = pd.DataFrame(date_and_view.T,columns = ['Date','Views'])
#         df["Artist"] = file_name[:len(file_name)-4].replace("-"," ")
#         artist_li.append(df)
#     # print("here")
#     # print(artist_li)
#     return artist_li
# # with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
# all_artist_data = processing_files(files_l, file_names)

    
# all_data = list(all_artist_data)
# all_data1 = [i for i in list(all_artist_data)]

# df2 = pd.concat(all_data1, ignore_index = True)

# print(df2)

    
        


In [15]:
# # with-out Threading


# from concurrent.futures import ThreadPoolExecutor
# import concurrent.futures

# my_lock = threading.Lock()


# def processing_files(files_l, file_names):
#     artist_li =[]
#     # print(files_l)
#     for file, file_name in zip(files_l, file_names):
#         df1 = pd.DataFrame()
#         #print(file)
#         date_and_view = np.load(file)
#         df = pd.DataFrame(date_and_view.T,columns = ['Date','Views'])
#         df["Artist"] = file_name[:len(file_name)-4].replace("-"," ")
#         artist_li.append(df)
#     # print("here")
#     # print(artist_li)
#     return artist_li
# # with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
# all_artist_data = processing_files(files_l, file_names)

# # print(type(all_artist_data))
# all_data = list(all_artist_data)
# all_data1 = [i for i in list(all_artist_data)]

# df2 = pd.concat(all_data1, ignore_index = True)

# print(df2)

    
        
